## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.hybrid.most_popular_user_CF import MostPopularCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9777705    ┆ 1655269 ┆ 990.0          ┆ 100.0      │
│ 9776152    ┆ 1999985 ┆ 59.0           ┆ 100.0      │
│ 9772706    ┆ 2447250 ┆ 171.0          ┆ 100.0      │
│ 9789721    ┆ 429452  ┆ 29.0           ┆ 100.0      │
│ 9784444    ┆ 75983   ┆ 26565.0        ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [3]:
recommender = MostPopularCollaborativeRecommender(train_df)
recommender.fit()

{1655269: [(1385974, np.float64(0.8398764717988548)),
  (84990, np.float64(0.8398764717988548)),
  (784484, np.float64(0.8386304226194501)),
  (2041948, np.float64(0.8377956234964408)),
  (2364990, np.float64(0.7564722399566952)),
  (1612478, np.float64(0.753697456201639)),
  (1250694, np.float64(0.7475005231409674)),
  (590692, np.float64(0.7151471862648273)),
  (1075966, np.float64(0.6392336120369814)),
  (1899524, np.float64(0.5384364042941386))],
 1999985: [(1282713, np.float64(0.8578481168496285)),
  (952062, np.float64(0.8578481168496285)),
  (2114301, np.float64(0.8578481168496285)),
  (418911, np.float64(0.857614073279152)),
  (1566174, np.float64(0.8573014734866659)),
  (1334444, np.float64(0.8539769002753843)),
  (2553831, np.float64(0.8512151462095306)),
  (486533, np.float64(0.8404705389665693)),
  (1445679, np.float64(0.7809775796392732)),
  (1866066, np.float64(0.7483287094146979))],
 2447250: [(2282200, np.float64(0.9325433653935147)),
  (899461, np.float64(0.93254336539

This first model just compares all artilces read by users when comparing users

In [4]:
binary_recommender = MostPopularCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{1655269: [(1268782, np.float64(0.40824829046386313)),
  (856038, np.float64(0.40824829046386313)),
  (1574760, np.float64(0.40824829046386313)),
  (299310, np.float64(0.40824829046386313)),
  (953177, np.float64(0.40824829046386313)),
  (1833813, np.float64(0.40824829046386313)),
  (1199397, np.float64(0.40824829046386313)),
  (1899524, np.float64(0.40824829046386313)),
  (1385974, np.float64(0.40824829046386313)),
  (1309962, np.float64(0.40824829046386313))],
 1999985: [(982009, np.float64(0.316227766016838)),
  (1828938, np.float64(0.2581988897471611)),
  (1427603, np.float64(0.2581988897471611)),
  (1703562, np.float64(0.22360679774997894)),
  (2517665, np.float64(0.22360679774997894)),
  (1017694, np.float64(0.22360679774997894)),
  (651414, np.float64(0.22360679774997894)),
  (1199732, np.float64(0.22360679774997894)),
  (2033355, np.float64(0.22360679774997894)),
  (269188, np.float64(0.22360679774997894))],
 2447250: [(1132651, np.float64(0.1867718419094071)),
  (389646, np.fl

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [5]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9782722, 9782131, 9783137, 9774297, 9776497]
reccomended for user  620796 :  [9771224, 9776967, 9772923, 9785835, 9757574]
reccomended for user  1067393 :  [9771113, 9785986, 9787230, 9671904, 9758825]
reccomended for user  1726258 :  [9771224, 9776967, 9772923, 9785835, 9783865]
reccomended for user  17205 :  [9780325, 9774764, 9780860, 9781991, 9776917]


In [6]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9774120, 9775484, 9773846, 9783334, 9782722]
reccomended for user  620796 :  [9783278, 9783334, 9771168, 9771686, 9769917]
reccomended for user  1067393 :  [9771113, 9774392, 9772355, 9771842, 9776916]
reccomended for user  1726258 :  [9771224, 9783278, 9771166, 9773846, 9773464]
reccomended for user  17205 :  [9775562, 9777804, 9779674, 9770729, 9777529]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [7]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.006161616161616161),
 'NDCG@K': np.float64(0.045849379491774755)}

The binary reccomender model only reccomending articles the user has not yet read

In [8]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.007037037037037037),
 'NDCG@K': np.float64(0.06373421134476696)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [9]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.005217391304347826),
 'NDCG@K': np.float64(0.037220095441632926)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [10]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.006666666666666668),
 'NDCG@K': np.float64(0.07005065503642185)}

## Model Experimentation

In [11]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9778500, 9786566, 9538375, 9789896, 9778351, 9780815, 9787441, 9778448, 9781875, 9772508, 9776855, 9786618, 9769531, 9780348, 9786719}
[9782722, 9782131, 9783137, 9774297, 9776497, 9786359, 9773574, 9778375, 9784662, 9779184, 9773282, 9778302, 9779517, 9788666, 9770288, 9790052, 9789703, 9785668, 9770102, 9782837, 9784591, 9778657, 9777856, 9779007, 9787586, 9776071, 9782635, 9785111, 9773846, 9774032, 9779737, 9782423, 9778381, 9788149, 9780020, 9778328, 9780325, 9772706, 9773985, 9773877, 9783646, 9774568, 9783197, 9784702, 9782234, 9771612, 9779860, 9779227, 9776234, 9780467, 9784870, 9771686, 9779204, 9785349, 9773486, 9783349, 9785019, 9782670, 9774120, 9788760, 9774352, 9777492, 9779045, 9785107, 9775331, 9775562, 9775776, 9780195, 9781086, 9771919, 9787465, 9790548, 9785475, 9779748, 9781476, 9787524, 9789810, 9780280, 9789065, 9783213, 9782092, 9782884, 9786378, 9775484, 9778168, 9786932, 9771916, 9784856, 9774079, 9782517, 9783057, 9778804, 9770082, 9776041, 9774074,

In [12]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9778500, 9786566, 9538375, 9789896, 9778351, 9780815, 9787441, 9778448, 9781875, 9772508, 9776855, 9786618, 9769531, 9780348, 9786719}
[9782722, 9782131, 9783137, 9774297, 9776497, 9786359, 9773574, 9778375, 9784662, 9779184, 9773282, 9778302, 9779517, 9788666, 9770288, 9790052, 9789703, 9785668, 9770102, 9782837, 9784591, 9778657, 9777856, 9779007, 9787586, 9776071, 9782635, 9785111, 9773846, 9774032, 9779737, 9782423, 9778381, 9788149, 9780020, 9778328, 9780325, 9772706, 9773985, 9773877, 9783646, 9774568, 9783197, 9784702, 9782234, 9771612, 9779860, 9776234, 9779227, 9780467, 9784870, 9771686, 9779204, 9785349, 9773486, 9783349, 9782670, 9785019, 9774120, 9788760, 9774352, 9777492, 9779045, 9775331, 9785107, 9775562, 9775776, 9780195, 9781086, 9771919, 9787465, 9790548, 9785475, 9779748, 9781476, 9787524, 9789810, 9780280, 9789065, 9783213, 9782092, 9782884, 9786378, 9775484, 9778168, 9786932, 9771916, 9784856, 9774079, 9782517, 9783057, 9778804, 9770082, 9776041, 9774074,

In [13]:
from utils.evaluation import perform_model_evaluation

matrics = perform_model_evaluation(recommender, test_df, k=5)
matrics

{'precision@k': np.float64(0.005913592651069327),
 'recall@k': np.float64(0.01215910418393617),
 'fpr@k': np.float64(0.0024630658654697713)}

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [14]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "user_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 15:49:31] [setup] RAM Tracking...
[codecarbon INFO @ 15:49:31] [setup] CPU Tracking...
[codecarbon WARNING @ 15:49:31] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon INFO @ 15:49:33] CPU Model on constant consumption mode: AMD Ryzen 5 5500U with Radeon Graphics
[codecarbon INFO @ 15:49:33] [setup] GPU Tracking...
[codecarbon INFO @ 15:49:34] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:49:34] >>> Tracker's metadata:
[codecarbon INFO @ 15:49:34]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 15:49:34]   Python version: 3.11.1
[codecarbon INFO @ 15:49:34]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 15:49:34]   Available RAM : 7.338 GB
[codecarbon INFO @ 15:49:34]   CPU count: 12
[codecarbon INFO @ 15:49:34]   CPU model: AMD Ryzen 5 5500U with Radeon Graphics
[codecarbon INFO @ 15:49:34]   GPU count: 1
[codecarbon INFO @ 15:49:34]   GPU model: 1 x NVIDIA GeForce GTX 1650
[codecarbon INFO @ 15:49:35] Saving emissions data to file c:\Users\chris\Desktop\NTNU Ting\8. Semester\Anbefalingssystemer\Project\TDT4215\recommender_system\demostrations\output\user_based_fit_emission.csv
[codecarbon INFO @ 15:49:50] Energy c

{'fit': ({1655269: [(1385974, np.float64(0.8398764717988548)),
    (84990, np.float64(0.8398764717988548)),
    (784484, np.float64(0.8386304226194501)),
    (2041948, np.float64(0.8377956234964408)),
    (2364990, np.float64(0.7564722399566952)),
    (1612478, np.float64(0.753697456201639)),
    (1250694, np.float64(0.7475005231409674)),
    (590692, np.float64(0.7151471862648273)),
    (1075966, np.float64(0.6392336120369814)),
    (1899524, np.float64(0.5384364042941386))],
   1999985: [(1282713, np.float64(0.8578481168496285)),
    (952062, np.float64(0.8578481168496285)),
    (2114301, np.float64(0.8578481168496285)),
    (418911, np.float64(0.857614073279152)),
    (1566174, np.float64(0.8573014734866659)),
    (1334444, np.float64(0.8539769002753843)),
    (2553831, np.float64(0.8512151462095306)),
    (486533, np.float64(0.8404705389665693)),
    (1445679, np.float64(0.7809775796392732)),
    (1866066, np.float64(0.7483287094146979))],
   2447250: [(2282200, np.float64(0.932543